In [1]:
# FUNCTIONS

# for MIR-QBSH query
def read_semitone_from_MIR_query(dir_query):
    list_query_name = []
    df_query = []
    truth = []

    for file in os.listdir(dir_query):
        if file.endswith(".csv"):
            list_query_name.append(file.replace(".csv",""))
            file_path = os.path.join(dir_query, file)
            fields = ["semitone"]
            temp_df = pd.read_csv(file_path, usecols=fields)
            df_query.append(temp_df.to_numpy().flatten())
            truth.append(file.split("-")[2])

    return list_query_name, df_query, truth

# for MIR-QBSH manually labelled query
def read_semitone_from_MIR_manual_query(dir_query):
    list_query_name = []
    df_query = []
    truth = []

    for file in os.listdir(dir_query):
        if file.endswith(".pv"):
            list_query_name.append(file.replace(".pv",""))
            file_path = os.path.join(dir_query, file)
            temp_df = pd.read_csv(file_path)
            temp_df = temp_df.to_numpy().flatten()
            round_df = []
            for i in temp_df:
                if i!=0:
                    round_df.append(round(i))
            df_query.append(round_df)
            truth.append(file.replace(".pv","").split("-")[2])

    return list_query_name, df_query, truth

# for IOACAS_QBH query
def read_semitone_from_IOACAS_query(dir_query):
    list_query_name = []
    df_query = []
    truth = []

    truth_file = os.path.join(dir_query, "query_truth.list")

    # read csv as string type need converters
    truth_df = pd.read_csv(truth_file, header=None, sep="\t", converters={i: str for i in range(100)})

    all_truth = truth_df[1].values.astype(str).flatten()
    # print("all truth", all_truth)
    all_wav = []

    for wav in truth_df[0]:
        all_wav.append(wav.split("\\")[1].replace(".wav",""))

    for file in os.listdir(dir_query):
        if file.endswith(".csv"):
            list_query_name.append(file.replace(".csv",""))
            file_path = os.path.join(dir_query, file)
            fields = ["semitone"]
            temp_df = pd.read_csv(file_path, usecols=fields)
            df_query.append(temp_df.to_numpy().flatten())
            # index_truth = all_wav.index("002_010")
            index_truth = all_wav.index(file.split("-")[0])
            truth.append(str(all_truth[index_truth]))
    # print("one truth", truth)

    return list_query_name, df_query, truth

# for MIR-QBSH and IOACAS-QBH database
def read_note_from_midi(dir_midi):
    pickled_semitone = "pickled_"+dir_midi.replace(folder_data+"\\","")
    dir_pickle = os.path.join(folder_pickle,pickled_semitone)

    if os.path.exists(dir_pickle):
        open_file = open(dir_pickle, "rb")
        list_dir_midi, df_midi = pickle.load(open_file)
        open_file.close()
    else:
        open_file = open(dir_pickle, "wb")
        list_dir_midi = []
        df_midi = []

        for file in os.listdir(dir_midi):
            if file.endswith(".csv"):
                list_dir_midi.append(file.replace(".mid.csv",""))
                file_path = os.path.join(dir_midi, file)
                fields = ["note_index"]
                temp_df = pd.read_csv(file_path, usecols=fields)
                df_midi.append(temp_df.to_numpy().flatten())
        pickle.dump([list_dir_midi, df_midi], open_file)
        open_file.close()

    return list_dir_midi, df_midi

# calculate relative distance
def calc_rel_dis(df):
    res = []
    length = len(df)
    for i in range(length-1):
        dis = float(df[i+1] - df[i])
        res.append(dis)

#     ada di eksperimen 1, tapi kayaknya nggak masuk akal kalau dipake
#     res = remove_consecutive(res)

    return res

# remove consecutive distance in list of relative distance 
def remove_consecutive(list, isQuery):
    if isQuery == True:
        if "DNN-LSTM" in dir_query:
            # eksperimen untuk DNN-LSTM
            list = get_only_n_consecutive_notes(list, 5)

        else:
            # eksperimen 1, 3
            # list = remove_only_consecutive(list)

            # eksperimen 2, 5, 6
            list = get_only_10_consecutive_notes(list)
                
            # eksperimen 4
            # list = get_exact_10_consecutive_notes(list)
        
    else:
        # eksperimen 2 dan 4
        # do nothing
        
        # eksperimen 1, 3, 5
        list = remove_only_consecutive(list)

    return list

import numpy as np
def remove_only_consecutive(ls):
    i = 0
    while i < len(ls)-1:
        if ls[i] == ls[i+1]:
            ls = np.delete(ls, i)
        else:
            i = i+1
    return ls

from itertools import groupby
def get_only_10_consecutive_notes(ls):
    res = []
    n = 10
    for k, v in groupby(ls):
        value = list(v)
        if len(value) >= n:
            res.append(value[0])
    return res

from itertools import groupby
def get_only_n_consecutive_notes(ls, n):
    res = []
    for k, v in groupby(ls):
        value = list(v)
        if len(value) >= n:
            res.append(value[0])
    return res

def get_exact_10_consecutive_notes(list):
    res = []
    i = 0
    while i < len(list)-9:
        cur = list[i]
        marker = True
        for j in range(1,10):
            if cur != list[i+j]:
                marker = False
                i+=j-1
                break
        if marker == True:
            res.append(cur)
            i+=10
        else:
            i+=1
            
    return res

# arrange index according to its similarity, more similar is less in index
def get_all_rank_max(ls):
    res = []
    for i in ls:
        temp = ls.index(max(ls))
        res.append(list_dir_midi[temp])
        ls[temp] = 0
    return res

# arrange index according to its distance, less distance is less in index
def get_all_rank_min(ls):
    res = []
    for i in ls:
        temp = ls.index(min(ls))
        res.append(list_dir_midi[temp])
        ls[temp] = sys.maxsize
    return res

# get rank from ground truth
def get_rank(rank, truth):
    try:
        return rank.index(truth)+1
    except ValueError as error:
        return sys.maxsize
    
# get top ten rank
def get_top_ten(rank):
    return rank[:10]

# get inverted index from all midi
def get_inverted_index_midi(dis_midi, list_dir_midi):
    # create inverted index for 2-grams, 3-grams, 4-grams in all midi
    import hashedindex
    RP4G = hashedindex.HashedIndex()
    RP3G = hashedindex.HashedIndex()
    RP2G = hashedindex.HashedIndex()

    # example:
    # midi relative distance : +1 +1 +4
    # 2-grams are +1 and +4
    # 3-grams are +1 +1 and +1 +3
    # 4-grams is +1 +1 +4

    # print(dis_midi[0])

    # inserting 2-grams as inverted index
    for midiNumber in range(len(dis_midi)):
        for note in dis_midi[midiNumber]:
            RP2G.add_term_occurrence(note, list_dir_midi[midiNumber])

    # test 2-grams
    # RP2G.get_documents((2.0))

    # inserting 3-grams as inverted index
    for midiNumber in range(len(dis_midi)):
        for noteNumber in range(len(dis_midi[midiNumber])-1):
            term = (dis_midi[midiNumber][noteNumber], dis_midi[midiNumber][noteNumber+1])
            RP3G.add_term_occurrence(term, list_dir_midi[midiNumber])

    # test 3-grams
    # RP3G.get_documents((2.0, 0.0))

    # inserting 4-grams as inverted index
    for midiNumber in range(len(dis_midi)):
        for noteNumber in range(len(dis_midi[midiNumber])-2):
            term = (dis_midi[midiNumber][noteNumber], 
            dis_midi[midiNumber][noteNumber+1], dis_midi[midiNumber][noteNumber+2])
            
            RP4G.add_term_occurrence(term, list_dir_midi[midiNumber])

    # test 4-grams
    # RP4G.get_documents((2.0, 0.0, -2.0))

    return RP4G, RP3G, RP2G

# get lists containing 4-grams, 3-grams, and 2-grams from each query
def get_ngrams_query(dis_query):
    RP4G = []
    RP3G = []
    RP2G = []

    # inserting list of 4-grams
    for noteNumber in range(len(dis_query)-2):
        term = (dis_query[noteNumber], dis_query[noteNumber+1], dis_query[noteNumber+2])
        RP4G.append(term)

    # inserting list of 3-grams
    for noteNumber in range(len(dis_query)-1):
        term = (dis_query[noteNumber], dis_query[noteNumber+1])
        RP3G.append(term)

    # inserting list of 2-grams
    RP2G = dis_query
    
    return RP4G, RP3G, RP2G

from collections import Counter
# get Counter containing number of query appearances in inverted index
def get_counter_grams(index, query):
    res = Counter()
    for grams in query:
        try:
            res += index.get_documents(grams)
        except IndexError as error:
            pass
    return res

# get rank from counter grams
def get_rank_from_counter_grams(res_grams, truth):
    rank = []
    for i in res_grams.most_common():
        rank.append(i[0])
    return rank

def get_rank_truth_from_counter_grams(rank, truth):
    try:
        rankTruth = get_rank(rank, truth)
    except ValueError as error:
        rankTruth = sys.maxsize
    return rankTruth

# get rank with relative pitch 4-grams (RP4G), 3-grams (RP3G) and 2-grams (RP2G)
def get_rank_with_rpg(dis_query, truth, index_midi):
    index4G, index3G, index2G = index_midi
    query4G, query3G, query2G = get_ngrams_query(dis_query)

    # 4-grams
    res_4grams = get_counter_grams(index4G, query4G)

    # 3-grams
    res_3grams = get_counter_grams(index3G, query3G)

    # 2-grams
    res_2grams = get_counter_grams(index2G, query2G)

    res_rank = []
    res_rankTruth = []

    rank = get_rank_from_counter_grams(res_4grams, truth)
    topTen = get_top_ten(rank)
    res_rank.append(topTen)
    res_rankTruth.append(get_rank_truth_from_counter_grams(rank, truth))

    rank = get_rank_from_counter_grams(res_3grams, truth)
    topTen = get_top_ten(rank)
    res_rank.append(topTen)
    res_rankTruth.append(get_rank_truth_from_counter_grams(rank, truth))

    rank = get_rank_from_counter_grams(res_2grams, truth)
    topTen = get_top_ten(rank)
    res_rank.append(topTen)
    res_rankTruth.append(get_rank_truth_from_counter_grams(rank, truth))

    return res_rankTruth, res_rank

def subsequent_MNF(ref, hyp):    
    res = 0
    if len(hyp)>len(ref):
        hyp, ref = ref, hyp
    
    diff = len(ref) - len(hyp)
    for i in range(diff+1):
        subRef = ref[i:i+len(hyp)]
        ratio = edit_distance.SequenceMatcher(a=subRef, b=hyp).ratio()
        if (res<ratio):
            res = ratio

    return res

# get rank with Mode Normalised Frequency using edit distance method
def get_rank_with_mnf(df_query, truth, midi_MNF):
    # ref = dis_midi
    # hyp = dis_query

    hyp = convert_df_to_MNF(df_query)

    list_ratio = []
    for ref in midi_MNF:
        sm = edit_distance.SequenceMatcher(a=ref, b=hyp)
        list_ratio.append(sm.ratio())
#         ratio = subsequent_MNF(ref, hyp)
#         list_ratio.append(ratio)
    
    rank = get_all_rank_max(list_ratio)
    rankTruth = get_rank(rank, truth)

    # show top ten result
    topTen = get_top_ten(rank)
    # print(topTen)

    return rankTruth, topTen

# get best rank from both nGrams and edit distance method
def get_rank_with_unified_algorithm(dis_query, df_query, truth, index_midi, midi_MNF):
    rank = []
    topTen = []

    temp_rank, temp_topTen = get_rank_with_rpg(dis_query, truth, index_midi)
    for r in temp_rank:
        rank.append(r)
    for t in temp_topTen:
        topTen.append(t)

    try:
        temp_rank, temp_topTen = get_rank_with_mnf(df_query, truth, midi_MNF)
        rank.append(temp_rank)
        topTen.append(temp_topTen)
    except ValueError as error:
        pass
    except IndexError as error:
        pass

    bestIndex = rank.index(min(rank))
    bestRank = rank[bestIndex]
    bestTopTen = topTen[bestIndex]

    # to return which algorithm: RP4G, RP3G, RP2G, or MNF
    switcher = {
        0: "RP4G",
        1: "RP3G",
        2: "RP2G",
        3: "MNF"
    }
    alg = switcher.get(bestIndex)

    return bestRank, bestTopTen, alg

# convert one dataframe containing semitone to Mode Normalised Frequency (MNF)
def convert_df_to_MNF(df):
    from collections import Counter
    res = []
    data = Counter(df)
    
    data = data.most_common()
    mode = data[0][0]
    adder = int(78-mode) # mode marked as char 'N' which is 78 in ascii

    for i in df:
        res.append(chr(int(i+adder)))
        # res.append(i) # to test matching with no MNF

    return res

# count MRR from a list of rank from some queries
def count_MRR(list_rank):
    mrr = 0
    counter = 0
    for rank in list_rank:
        if not rank == sys.maxsize:
            mrr+=1/rank
            counter+=1
    return round(mrr/counter,2), counter

# count top 1 hit ratios from list rank
def count_top_1_ratio(list_rank):
    count_top1 = 0
    counter = 0
    for rank in list_rank:
        if not rank == sys.maxsize:
            if rank == 1:
                count_top1+=1
            counter+=1
    return round(count_top1/counter,2), counter

# count top 3 hit ratios from list rank
def count_top_3_ratio(list_rank):
    count_top3 = 0
    counter = 0
    for rank in list_rank:
        if not rank == sys.maxsize:
            if rank <= 3:
                count_top3+=1
            counter+=1
    return round(count_top3/counter,2), counter

# count top 5 hit ratios from list rank
def count_top_5_ratio(list_rank):
    count_top5 = 0
    counter = 0
    for rank in list_rank:
        if not rank == sys.maxsize:
            if rank <= 5:
                count_top5+=1
            counter+=1
    return round(count_top5/counter,2), counter

# count top 10 hit ratios from list rank
def count_top_10_ratio(list_rank):
    count_top10 = 0
    counter = 0
    for rank in list_rank:
        if not rank == sys.maxsize:
            if rank <= 10:
                count_top10+=1
            counter+=1
    return round(count_top10/counter,2), counter

def print_result_to_file(process_name):
    from datetime import datetime

    dateTimeObj = str(datetime.now())+".txt"
    dateTimeObj = dateTimeObj.replace(":","-")
    filename = os.path.join(folder_hasil, process_name + ' ' + dateTimeObj)
    with open(filename, 'w') as f:
        f.write(cap.stdout)

from tslearn.metrics import dtw_subsequence_path as dtw
def get_rank_with_dtw(df_query, df_midi, truth):
    list_dist = []
    
    if "DNN-LSTM" in dir_query:
        # eksperimen untuk DNN-LSTM
        query = get_only_n_consecutive_notes(df_query, 5)

    else:
        # eksperimen 6
        query = get_only_10_consecutive_notes(df_query)

    if len(query)==0:
        return sys.maxsize

    for midi in df_midi:
        midi = remove_only_consecutive(midi)
        path, dist = dtw(query, midi)
        list_dist.append(dist)

    rank = get_all_rank_min(list_dist)

    rankTruth = get_rank(rank, truth)

    return rankTruth

def do_matching_process(dis_query, c_query, truth, index_midi, midi_MNF):
    list_rank = []
    list_time = []

    for i in range(len(c_query)):
        print("Processing...",i+1,"/",len(c_query))
        
        start_time = time.time()

        # eksperimen 1,2,3,4,5
        rankTruth, topTen, alg = get_rank_with_unified_algorithm(dis_query[i], c_query[i], truth[i], index_midi, midi_MNF)

        # eksperimen 6
        # rankTruth = get_rank_with_dtw(c_query[i], df_midi, truth[i])

        # show ground truth from a query
        # print("query",list_query_name[i],"truth is",truth[i])

        found = rankTruth != sys.maxsize

        if found == True:
            list_time.append(round(time.time() - start_time, 3))
            list_rank.append(rankTruth)

            print("query",list_query_name[i],"truth is on rank",rankTruth)
            # print("top ten result from",alg,":",topTen)
        else:
            print("query",list_query_name[i],"truth is not found on database")

    return list_rank, list_time

def process_query(dir_query, dir_midi, list_query_name, df_query, truth, list_dir_midi, df_midi):
    # get compressed query and midi by removing consecutive notes
    c_query = []
    for query in df_query:
        compressed_query = remove_consecutive(query, isQuery = True)
        c_query.append(compressed_query)

    c_midi = []
    for midi in df_midi:
        compressed_midi = remove_consecutive(midi, isQuery=False)
        c_midi.append(compressed_midi)

    # calculate relative distance in all query and midi
    dis_query, dis_midi = get_rel_dis_query_midi(c_query, c_midi)

    index_midi = get_inverted_index_midi(dis_midi, list_dir_midi)

    midi_MNF = []
    for i in c_midi:
        temp = convert_df_to_MNF(i)
        midi_MNF.append(temp)

    print("Start matching process")

    list_rank, list_time = do_matching_process(dis_query, c_query, truth, index_midi, midi_MNF)

    try:
        mrr, counter = count_MRR(list_rank)
        print("MRR:", mrr, "from", counter, "queries")

        hit_ratio, counter = count_top_1_ratio(list_rank)
        print("Top 1 ratio:", hit_ratio, "from", counter, "queries" )

        hit_ratio, counter = count_top_3_ratio(list_rank)
        print("Top 3 ratio:", hit_ratio, "from", counter, "queries" )

        hit_ratio, counter = count_top_5_ratio(list_rank)
        print("Top 5 ratio:", hit_ratio, "from", counter, "queries" )

        hit_ratio, counter = count_top_10_ratio(list_rank)
        print("Top 10 ratio:", hit_ratio, "from", counter, "queries" )
        
        print("Average time for matching:", round(sum(list_time)/len(list_time), 3),"s")
    
    except ZeroDivisionError:
        print("No result to show")
        # print("list rank", list_rank)
        # print("list time", list_time)

    print("Finish matching process")

def process_query_uncontinue(dir_query, dir_midi, list_query_name, df_query, truth, list_dir_midi, df_midi):
    df_query_firstHalf = []
    df_query_secondHalf = []
    
    for query in df_query:
        firstHalf = query[:len(query)//2]
        secondHalf = query[len(query)//2:]
        df_query_firstHalf.append(firstHalf)
        df_query_secondHalf.append(secondHalf)

    # get compressed query and midi by removing consecutive notes
    c_query_firstHalf = []
    for query in df_query_firstHalf:
        compressed_query = remove_consecutive(query, isQuery = True)
        c_query_firstHalf.append(compressed_query)

    c_query_secondHalf = []
    for query in df_query_secondHalf:
        compressed_query = remove_consecutive(query, isQuery = True)
        c_query_secondHalf.append(compressed_query)

    c_midi = []
    for midi in df_midi:
        compressed_midi = remove_consecutive(midi, isQuery=False)
        c_midi.append(compressed_midi)

    # calculate relative distance in all query and midi
    dis_query_firstHalf, dis_midi = get_rel_dis_query_midi(c_query_firstHalf, c_midi)
    dis_query_secondHalf, dis_midi = get_rel_dis_query_midi(c_query_secondHalf, c_midi)

    index_midi = get_inverted_index_midi(dis_midi, list_dir_midi)

    midi_MNF = []
    for i in c_midi:
        temp = convert_df_to_MNF(i)
        midi_MNF.append(temp)

    print("Start matching process")

    print("Start matching for first half")
    list_rank_firstHalf, list_time_firstHalf = do_matching_process(dis_query_firstHalf, c_query_firstHalf, truth, index_midi, midi_MNF)
    print("Finish matching for first half")

    print("Start matching for second half")
    list_rank_secondHalf, list_time_secondHalf = do_matching_process(dis_query_secondHalf, c_query_secondHalf, truth, index_midi, midi_MNF)
    print("Finish matching for second half")

    list_best_rank = []
    list_total_time = []

    for counter in range(len(list_rank_firstHalf)):
        if list_rank_firstHalf[counter]>list_rank_secondHalf[counter]:
            temp_rank = list_rank_secondHalf[counter]
        elif list_rank_firstHalf[counter]<list_rank_secondHalf[counter]:
            temp_rank = list_rank_firstHalf[counter]
        else:
            temp_rank = list_rank_firstHalf[counter]

        total_time = list_time_firstHalf[counter]+list_time_secondHalf[counter]

        list_best_rank.append(temp_rank)
        list_total_time.append(total_time)

    try:
        mrr, counter = count_MRR(list_best_rank)
        print("MRR:", mrr, "from", counter, "queries")

        hit_ratio, counter = count_top_1_ratio(list_best_rank)
        print("Top 1 ratio:", hit_ratio, "from", counter, "queries" )

        hit_ratio, counter = count_top_3_ratio(list_best_rank)
        print("Top 3 ratio:", hit_ratio, "from", counter, "queries" )

        hit_ratio, counter = count_top_5_ratio(list_best_rank)
        print("Top 5 ratio:", hit_ratio, "from", counter, "queries" )

        hit_ratio, counter = count_top_10_ratio(list_best_rank)
        print("Top 10 ratio:", hit_ratio, "from", counter, "queries" )
        
        print("Average time for matching:", round(sum(list_total_time)/len(list_total_time), 3),"s")
    
    except ZeroDivisionError:
        print("No result to show")
        # print("list rank", list_best_rank)
        # print("list time", list_total_time)

    print("Finish matching process")

import math
# convert pitch to semitone
def convert_f0_to_semitone(f0):
    res = []
    for freq in f0:
        res.append(round(12*math.log((freq/440),2)+69))
    return res

# extract pitch and semitone with parselmouth from wav to csv
def extract_pitch_with_parselmouth_to_csv(folder, dir_out):
    import parselmouth
    import numpy as np
    import os
    import pandas as pd

    for file in os.listdir(folder):
        if file.endswith(".wav"):
            file_path = os.path.join(folder, file)

            snd = parselmouth.Sound(file_path)

            pitch = snd.to_pitch()
            pitch_values = pitch.selected_array['frequency']

            time = []
            f0 = []

            for i in range(len(pitch_values)):
                if pitch_values[i]!=0:
                    f0.append(pitch_values[i].round(decimals=3))
                    time.append(pitch.xs()[i].round(decimals=3))

            file = file_path.split("\\")
            file_csv = os.path.join(dir_out, file[2] + "-parselmouth.csv")
            file_csv = file_csv.replace(".wav","")

            semitone = convert_f0_to_semitone(f0)

            pd.DataFrame({'time':time, 'semitone':semitone, 'f0':f0}).to_csv(file_csv, index=False)

# extract pitch and semitone with DNN-LSTM from wav to csv
def extract_pitch_with_DNN_LSTM_to_csv(folder, dir_out):
    import shutil
    
    folder_project_DNN_LSTM = "Lab_MelExt"

    dir_temp_in = os.path.join(folder_project_DNN_LSTM, "temp_query")

    if not os.path.exists(dir_temp_in):
        os.mkdir(dir_temp_in)
    else:
        delete_folder_content(dir_temp_in)

    # copy query from main folder to DNN-LSTM project folder
    for i in os.listdir(folder):
        source = os.path.join(folder, i)
        destination = os.path.join(dir_temp_in, i)
        shutil.copy(source, destination)

    dir_temp_query = os.path.join(folder_project_DNN_LSTM, "temp_pitch")
    
    if os.path.exists(dir_temp_query):
        delete_folder_content(dir_temp_query)

    cwd = os.getcwd()
    os.chdir(folder_project_DNN_LSTM)
    cmd_line = "python predict.py temp_query temp_pitch"
    os.system(cmd_line)
    os.chdir(cwd)

    for file in os.listdir(dir_temp_query):
        if file.endswith(".csv"):
            filePath = os.path.join(dir_temp_query, file)
            df = pd.read_csv(filePath, sep="\t", header=None)

            time = []
            f0 = []

            for i in range(len(df[1])):
                if df[1][i]!=0:
                    time.append(df[0][i].round(decimals=3))
                    f0.append(df[1][i].round(decimals=3))
                    
            semitone = convert_f0_to_semitone(f0)

            file_csv = os.path.join(dir_out, file)

            pd.DataFrame({'time':time, 'semitone':semitone, 'f0':f0}).to_csv(file_csv, index=False)
    
    shutil.rmtree(dir_temp_in)
    shutil.rmtree(dir_temp_query)

def extract_pitch_to_csv(extractor, folder, dir_out):
    print("Start melody extraction with",extractor)
    start_time = time.time()
    if extractor == "parselmouth":
        extract_pitch_with_parselmouth_to_csv(folder, dir_out)
    elif extractor == "DNN-LSTM":
        extract_pitch_with_DNN_LSTM_to_csv(folder, dir_out)
    duration = round(time.time() - start_time, 3)
    print("Average time for melody extraction:",round(duration/query_count,3),"s")
    print("Finish melody extraction with",extractor)

# choose midi based on dir_query
def get_midi_dir(dir_query):
    if "48" in dir_query:
        dir_midi = os.path.join(folder_data, "Database midi IOACAS-QBH 48 MIDI")
    elif "IOACAS" in dir_query:
        dir_midi = os.path.join(folder_data, "Database midi IOACAS-QBH")
    elif "Putri" in dir_query:
        dir_midi = os.path.join(folder_data, "Database midi Putri")
    else:
        dir_midi = os.path.join(folder_data, "Database midi MIR-QBSH")
    
    return dir_midi

# read and insert semitone from query to list of dataframe
def get_semitone_list(dir_query):
    pickled_semitone = "pickled_"+dir_query.replace(folder_data+"\\","")
    dir_pickle = os.path.join(folder_pickle,pickled_semitone)

    if os.path.exists(dir_pickle):
        open_file = open(dir_pickle, "rb")
        list_query_name, df_query, truth = pickle.load(open_file)
        open_file.close()
    else:
        if "manual" in dir_query:
            list_query_name, df_query, truth = read_semitone_from_MIR_manual_query(dir_query)
        elif "IOACAS" in dir_query:
            list_query_name, df_query, truth = read_semitone_from_IOACAS_query(dir_query)
        elif "Putri" in dir_query:
            list_query_name, df_query, truth = read_semitone_from_IOACAS_query(dir_query)
        else:
            list_query_name, df_query, truth = read_semitone_from_MIR_query(dir_query)
        if not any(value in dir_query for value in ("Test", "test", "Temp", "temp", "Demo", "demo")):
            open_file = open(dir_pickle, "wb")
            pickle.dump([list_query_name, df_query, truth], open_file)
            open_file.close()

    return list_query_name, df_query, truth

# calculate relative distance in all query and midi
def get_rel_dis_query_midi(df_query, df_midi):
    # calculate relative distance in all query
    dis_query = []
    for query in df_query:
        dis_query.append(calc_rel_dis(query))

    # calculate relative distance in all midi
    dis_midi = []
    for midi in df_midi:
        dis_midi.append(calc_rel_dis(midi))

    return dis_query, dis_midi

def save_to_wav(snd, filename):
    if ".wav" not in filename:
        filename = filename + ".wav"
    snd.save(filename, "WAV")

# code taken from https://dspillustrations.com/pages/posts/misc/fourier-series-and-harmonic-approximation.html
def fourierSeries(period, N):
    """Calculate the Fourier series coefficients up to the Nth harmonic"""
    result = []
    T = len(period)
    t = np.arange(T)
    for n in range(N+1):
        an = 2/T*(period * np.cos(2*np.pi*n*t/T)).sum()
        bn = 2/T*(period * np.sin(2*np.pi*n*t/T)).sum()
        result.append((an, bn))
    return np.array(result)

# code taken from https://dspillustrations.com/pages/posts/misc/fourier-series-and-harmonic-approximation.html
def reconstruct(P, anbn):
    result = 0
    t = np.arange(P)
    for n, (a, b) in enumerate(anbn):
        if n == 0:
            a = a/2
        result += a*np.cos(2*np.pi*n*t/P) + b * np.sin(2*np.pi*n*t/P)
    return result

import numpy as np
def compute_FSD(data):
    # assume sample rate = 64000
    # with n_frame = 640, each frame will contain 100 samples
    # with n_harmonics = 10, each frame will reduce from 100 samples to 10 harmonics
    n_frame = 640
    n_harmonics = 10

    # Framing
    data_split = np.array_split(data,n_frame)

    new_data = np.array([])
    
    for frame in data_split:
        # Fourier series Expansion with N order
        F = fourierSeries(frame, n_harmonics)

        # Frame reconstruction
        frame_i = reconstruct(len(frame), F[:n_harmonics,:])

        # Signal reconstruction
        new_data = np.concatenate((new_data, frame_i), axis=None)

    return new_data

import parselmouth
def remove_noise_with_FSD(snd):
    data = snd.values[0]
    samplerate = snd.sampling_frequency

    clean_data = compute_FSD(data)
    clean_snd = parselmouth.Sound(values=clean_data,sampling_frequency=samplerate)

    return clean_snd

# SS v3.0
import parselmouth
def remove_noise_with_spectral_subtraction(snd):
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']

    final_unvoiced_length = 0
    final_unvoiced_start_time = 0.0
    final_unvoiced_end_time = 0.0
    found_unvoiced = False
    
    for i in range(len(pitch_values)-1):
        temp_unvoiced_length = 0
        temp_unvoiced_start_time = pitch.xs()[i].round(decimals=3)

        while pitch_values[i] == 0.0 and i < len(pitch_values)-1:
            temp_unvoiced_length += 1
            i += 1
        
        temp_unvoiced_end_time = pitch.xs()[i].round(decimals=3)

        bandwidth = temp_unvoiced_end_time - temp_unvoiced_start_time
        # batas minimal bandwidth praat = 0.0625 (dalam detik)

        if temp_unvoiced_length > final_unvoiced_length:
            if temp_unvoiced_start_time <= 0.1 and bandwidth > 0.5 :
                final_unvoiced_length = temp_unvoiced_length
                final_unvoiced_start_time = temp_unvoiced_start_time
                final_unvoiced_end_time = temp_unvoiced_end_time
                found_unvoiced = True
                break
            elif found_unvoiced == False:
                final_unvoiced_length = temp_unvoiced_length
                final_unvoiced_start_time = temp_unvoiced_start_time
                final_unvoiced_end_time = temp_unvoiced_end_time

    if final_unvoiced_length!=0:
        # snd_unvoiced = snd.extract_part(from_time = final_unvoiced_start_time, to_time = final_unvoiced_end_time, 
        # preserve_times=True)
        
        snd_noise_removed = parselmouth.praat.call(snd, "Remove noise", final_unvoiced_start_time, 
        final_unvoiced_end_time, 0.025, 80.0, 10000.0, 40.0, "Spectral subtraction")

        parselmouth.Vector.scale_peak(snd_noise_removed)

        # return parselmouth object
        return snd_noise_removed
    else:
        print("no unvoiced segment found for", filename)
        return snd

def clean_wav_file(folder, dir_out):
    print("Start cleaning wav file")
    start_time = time.time()
    for file in os.listdir(folder):
        if file.endswith(".wav"):
            file_in = os.path.join(folder, file)
            snd_object = parselmouth.Sound(file_in)
            snd_object = remove_noise_with_FSD(snd_object)
            snd_object = remove_noise_with_spectral_subtraction(snd_object)
            file_out = file_in.replace(folder,dir_out)
            file_out = file_out.replace(".wav","-clean.wav")
            save_to_wav(snd_object, file_out)
    duration = round(time.time() - start_time, 3)
    print("Average time for cleaning:",round(duration/query_count,3),"s")
    print("Finish cleaning wav file")

import os
def delete_folder_content(folder):
    content = os.listdir(folder)
    for file in content:
        filename = os.path.join(folder,file)
        os.remove(filename)

import shutil
def move_IOACAS_ground_truth_list_file(source, destination):
    source = os.path.join(source, "query_truth.list")
    destination = os.path.join(destination, "query_truth.list")
    shutil.copy(source, destination)

def make_uncontinue_query(list_query_name, df_query, truth):
    df_query_uncontinue = []
    for query in df_query:
        temp_query = []
        size = len(query)
        oneThirdSize = int(size/3)
        twoThirdSize = 2*oneThirdSize
        for q in query[:oneThirdSize]:
            temp_query.append(q)
        for q in query[twoThirdSize:size]:
            temp_query.append(q)
        df_query_uncontinue.append(temp_query)
    print("uncontinue query")
    return list_query_name, df_query_uncontinue, truth

In [ ]:
# %%capture cap --no-stderr

# this cell is for matching only

# read midi and query data
# insert it into dataframe

import os
import pandas as pd
import edit_distance
import time
import sys
import pickle

folder_data = "Data query dan midi"
folder_hasil = "Hasil eksperimen"
folder_pickle = "pickle"

if not os.path.exists(folder_hasil):
    os.mkdir(folder_hasil)

if not os.path.exists(folder_pickle):
    os.mkdir(folder_pickle)

# -------------test query-------------
dir_query = os.path.join(folder_data, "Test query MIR-QBSH_midi-to-wav_parselmouth")
# dir_query = os.path.join(folder_data, "Test query MIR-QBSH_midi-to-wav_DNN-LSTM")
# dir_query = os.path.join(folder_data, "Test query MIR-QBSH_parselmouth")
# dir_query = os.path.join(folder_data, "Test query MIR-QBSH_DNN-LSTM")
# dir_query = os.path.join(folder_data, "Test query MIR-QBSH_manual label")

# dir_query = os.path.join(folder_data, "Test query Putri_parselmouth")
# dir_query = os.path.join(folder_data, "Test query Putri_vokal sama nada_parselmouth")
# dir_query = os.path.join(folder_data, "Test query Putri_vokal beda nada_parselmouth")

# dir_query = os.path.join(folder_data, "Test query IOACAS_QBH_48_MIDI_midi to wav_parselmouth")
# dir_query = os.path.join(folder_data, "Test query IOACAS_QBH_parselmouth")
# dir_query = os.path.join(folder_data, "Test query IOACAS_QBH_DNN-LSTM")
# dir_query = os.path.join(folder_data, "Test query IOACAS_QBH_parselmouth_48_MIDI")
# dir_query = os.path.join(folder_data, "Test query IOACAS_QBH_DNN-LSTM_48_MIDI")


# -------------all query-------------
# dir_query = os.path.join(folder_data, "Query_MIR_QBSH_parselmouth")
# dir_query = os.path.join(folder_data, "Query_MIR_QBSH_DNN-LSTM")
# dir_query = os.path.join(folder_data, "Query_MIR_QBSH_manual label")
# dir_query = os.path.join(folder_data, "Query_IOACAS_QBH_parselmouth")
# dir_query = os.path.join(folder_data, "Query_IOACAS_QBH_DNN-LSTM")
# dir_query = os.path.join(folder_data, "Query_IOACAS_QBH_parselmouth_48_MIDI")
# dir_query = os.path.join(folder_data, "Query_IOACAS_QBH_DNN-LSTM_48_MIDI")

# choose midi based on dir_query
dir_midi = get_midi_dir(dir_query)

process_name = dir_query.replace(folder_data+"\\","")+" with "+dir_midi.replace(folder_data+"\\","")
print("Start process", process_name)

# read and insert semitone from query to list of dataframe
list_query_name, df_query, truth = get_semitone_list(dir_query)

# makes the query discontinuous
list_query_name, df_query, truth = make_uncontinue_query(list_query_name, df_query, truth)

# read and insert note_index or semitone from midi to list of dataframe
list_dir_midi, df_midi = read_note_from_midi(dir_midi)

# test match with query data
# process_query(dir_query, dir_midi, list_query_name, df_query, truth, list_dir_midi, df_midi)
process_query_uncontinue(dir_query, dir_midi, list_query_name, df_query, truth, list_dir_midi, df_midi)

print("Finish process", process_name)

# place on different cell
# print_result_to_file(process_name)

In [3]:
# %%capture cap --no-stderr

# FOR DEMO PURPOSES
# COMBINED ALL PROCESS FROM NOISE FILTRATION, PITCH EXTRACTION, AND MATCHING
# PARSELMOUTH AND DNN-LSTM PITCH EXTRACTION

import os
import pandas as pd
import edit_distance
import time
import sys
import shutil
import pickle

folder_data = "Data query dan midi"
folder_hasil = "Hasil eksperimen"
folder_pickle = "pickle"
# folder_wav = "Wav_Query_MIR_QBSH_test"
# folder_wav = "Wav_Query_IOACAS_QBH_48_MIDI_test"
# folder_wav = "Wav_Query_IOACAS_QBH_test"

# folder_wav = "Wav_Query_MIR_QBSH"
# folder_wav = "Wav_Query_IOACAS_QBH_48_MIDI"
# folder_wav = "Wav_Query_IOACAS_QBH"
folder_wav = "Wav_Query_MIR_QBSH_test"

# PARAMETER
# use noise filtration or not
do_noise_filtration = True
# do_noise_filtration = False

# pitch extractor
extractor = "parselmouth"
# extractor = "DNN-LSTM"

if not os.path.exists(folder_hasil):
    os.mkdir(folder_hasil)

dir_wav_query = os.path.join(folder_data, folder_wav)

if "IOACAS" in dir_wav_query:
    query_count = len(os.listdir(dir_wav_query))-1
else:
    query_count = len(os.listdir(dir_wav_query))

# choose midi based on dir_query
dir_midi = get_midi_dir(dir_wav_query)

process_name = dir_wav_query.replace(folder_data+"\\","")+" with "+dir_midi.replace(folder_data+"\\","")
print("Start process", process_name)

print("Number of queries:", query_count)

# start query cleaning or noise filtration

if do_noise_filtration == True:
    dir_clean_wav_query = dir_wav_query+"_clean"

    if not os.path.exists(dir_clean_wav_query):
        os.mkdir(dir_clean_wav_query)
    else:
        delete_folder_content(dir_clean_wav_query)

    clean_wav_file(dir_wav_query, dir_clean_wav_query)

    # copy IOACAS ground truth list file
    if "IOACAS" in dir_wav_query:
        move_IOACAS_ground_truth_list_file(dir_wav_query, dir_clean_wav_query)

# end query cleaning or noise filtration

# start pitch extraction

if do_noise_filtration == True:
    dir_wav_query = dir_clean_wav_query

dir_query = dir_wav_query+"_"+extractor

if not os.path.exists(dir_query):
    os.mkdir(dir_query)
else:
    delete_folder_content(dir_query)

extract_pitch_to_csv(extractor, dir_wav_query, dir_query)

# copy IOACAS ground truth list file
if "IOACAS" in dir_wav_query:
    move_IOACAS_ground_truth_list_file(dir_wav_query, dir_query)
    
# end pitch extraction

# start matching

# read and insert semitone from query to list of dataframe
list_query_name, df_query, truth = get_semitone_list(dir_query)

# read and insert note_index or semitone from midi to list of dataframe
list_dir_midi, df_midi = read_note_from_midi(dir_midi)

# start query processing and matching with database
process_query(dir_query, dir_midi, list_query_name, df_query, truth, list_dir_midi, df_midi)

# end matching

print("Finish process", process_name)

# place on different cell
# print_result_to_file(process_name)

Start process Wav_Query_MIR_QBSH_test with Database midi MIR-QBSH
Number of queries: 2
Start cleaning wav file
Average time for cleaning: 1.145 s
Finish cleaning wav file
Start melody extraction with parselmouth
Average time for melody extraction: 0.212 s
Finish melody extraction with parselmouth
Start matching process
Processing... 1 / 2
query year2003-person00001-00013-clean-parselmouth truth is on rank 12
Processing... 2 / 2
query year2003-person00001-00014-clean-parselmouth truth is on rank 15
MRR: 0.07 from 2 queries
Top 1 ratio: 0.0 from 2 queries
Top 3 ratio: 0.0 from 2 queries
Top 5 ratio: 0.0 from 2 queries
Top 10 ratio: 0.0 from 2 queries
Average time for matching: 0.261 s
Finish matching process
Finish process Wav_Query_MIR_QBSH_test with Database midi MIR-QBSH
